In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split

In [84]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False, parse_dates = ['timestamp'], infer_datetime_format = True,
                    dtype = {'event': 'category','condition': 'category','storage': 'category', 'color': 'category', 'staticpage': 'category', 'campaign_source': 'category', 'search_engine': 'category', 'channel': 'category', 'new_vs_returning': 'category', 'region': 'category', 'country': 'category', 'device_type': 'category'})
labels = pd.read_csv('labels_training_set.csv', low_memory = False).set_index('person')

In [128]:
dfPredictor = pd.DataFrame(df['person'].unique())
dfPredictor.columns = ['person']
dfPredictor = dfPredictor.set_index('person')

ult15Dias = df.loc[df['timestamp'] > pd.to_datetime('15-05-2018')]

checkoutsPorPersona = ult15Dias.loc[ult15Dias['event'] == 'checkout']['person'].value_counts().to_frame()
checkoutsPorPersona.columns = ['checkoutsUlt15Mayo']

dfPredictor = dfPredictor.join(checkoutsPorPersona, how = 'left', on = 'person').fillna(value = 0)
dfPredictorLocal = dfPredictor.join(labels, how = 'inner')

In [129]:
X_train, X_test, y_train, y_test = train_test_split(dfPredictorLocal.iloc[:,:-1], dfPredictorLocal.iloc[:,-1:], test_size=0.2, random_state=123)

In [130]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                            colsample_bytree = 0.3, learning_rate = 0.1,
                            max_depth = 5, alpha = 10, n_estimators = 10)

In [131]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
preds = xg_reg.predict(X_test)

In [133]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

0.816328267063411

In [134]:
predicTodo0 = np.zeros(3883)
roc_auc_score(y_test, predicTodo0)

0.5

In [157]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv', low_memory = False)
kaggle = kaggle.set_index('person')
dfPredictorSubmit = dfPredictor.join(kaggle, how = 'inner')

In [158]:
predsSubmit = xg_reg.predict(dfPredictorSubmit)

In [159]:
dfPredictorSubmit['label'] = predsSubmit
#paraExport = pd.DataFrame(predsSubmit)
#paraExport

In [162]:
dfPredictorSubmit.drop(axis = 1, labels = 'checkoutsUlt15Mayo').to_csv('submit.csv')
